### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [167]:
import warnings
warnings.filterwarnings(action='ignore')

In [168]:
import mlflow
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature

In [169]:
#reading dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [170]:
data.shape

(4898, 12)

In [171]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [172]:
data['quality'].unique()

array([6, 5, 7, 8, 4, 3, 9])

In [173]:
#split data into train test split
train,test=train_test_split(data,test_size=0.25,random_state=42)

In [174]:
train.shape

(3673, 12)

In [175]:
test.shape

(1225, 12)

In [176]:
train['quality']


2835    6
1157    6
744     5
1448    6
3338    6
       ..
4426    6
466     6
3092    6
3772    6
860     8
Name: quality, Length: 3673, dtype: int64

In [177]:
train[['quality']].values

array([[6],
       [6],
       [5],
       ...,
       [6],
       [6],
       [8]])

In [178]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8])

In [179]:
#training data
train_x=train.drop(['quality'],axis=1)
train_y=train['quality'].values.ravel()
#test data

test_x=test.drop(['quality'],axis=1)
test_y=test['quality'].values.ravel()

#further splitting train data into train and validation data
train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.2,random_state=42)

In [180]:
signature=infer_signature(train_x,train_y)

In [181]:
train_x

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
204,5.8,0.28,0.35,2.3,0.053,36.0,114.0,0.99240,3.28,0.50,10.2
2413,7.1,0.21,0.28,2.7,0.034,23.0,111.0,0.99405,3.35,0.64,10.2
4832,5.9,0.32,0.26,1.5,0.057,17.0,141.0,0.99170,3.24,0.36,10.7
623,7.2,0.24,0.34,1.1,0.045,3.0,64.0,0.99130,3.23,0.51,11.4
3509,6.4,0.26,0.25,10.7,0.046,66.0,179.0,0.99606,3.17,0.55,9.9
...,...,...,...,...,...,...,...,...,...,...,...
22,6.8,0.26,0.42,1.7,0.049,41.0,122.0,0.99300,3.47,0.48,10.5
509,6.0,0.24,0.27,1.9,0.048,40.0,170.0,0.99380,3.64,0.54,10.0
2640,7.4,0.20,0.37,1.2,0.028,28.0,89.0,0.99132,3.14,0.61,11.8
1733,8.1,0.30,0.31,1.1,0.041,49.0,123.0,0.99140,2.99,0.45,11.1


In [182]:
# np.array(np.mean(train_x,axis=0))
np.mean(train_x,axis=0)

fixed acidity             6.866219
volatile acidity          0.280378
citric acid               0.332597
residual sugar            6.421647
chlorides                 0.045551
free sulfur dioxide      35.355684
total sulfur dioxide    138.792376
density                   0.994074
pH                        3.189193
sulphates                 0.488397
alcohol                  10.500567
dtype: float64

In [183]:
#ANN Model
import mlflow.tensorflow


def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    #mean and standard devaiation for normalization
    mean=np.array(np.mean(train_x,axis=0))
    var=np.array(np.var(train_x,axis=0))


    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    #compile the model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['lr'],momentum=params['momentum']),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()])
    
    #train the model with params lr and momentum with mlflow tracking

    with mlflow.start_run(nested=True):

        model.fit(train_x,train_y,validation_data=(valid_x,valid_y)
                  ,batch_size=64,
                  epochs=epochs)
    
        #evaluate the model
        eval_result=model.evaluate(val_x,val_y,batch_size=64)
        eval_rmse=eval_result[1]

        #log the parameter and result
        mlflow.log_params(params=params)
        mlflow.log_metric('eval rmse',eval_rmse)

        #log the model
        mlflow.tensorflow.log_model(model,artifact_path='model')
        
        return {'loss':eval_rmse,'status':STATUS_OK,'model':model}

        
    


In [184]:
def objective(params):
    result=train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=val_x,
        valid_y=val_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [185]:
space ={
    'lr':hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    'momentum':hp.uniform('momentum',0.0,1.0)
}

In [186]:
import mlflow.tensorflow


mlflow.set_experiment('wine-quality-v2')
with mlflow.start_run():
    #conduct the hyperparameter search using Hyperopt
    trails=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trails
    )

    #fetch the details of the best run
    best_run=sorted(trails.results,key=lambda x : x['loss'])[0]

    #log the best parameter , loss , and model
    mlflow.log_params(best)
    mlflow.log_metric('eval rmse',best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],'model')
    

    #print out the best paraymeter and corresponding loss

    print(f"best parameter: {best}")
    print(f"best eval rmse : {best_run['loss']}")    

2025/06/07 11:16:58 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality-v2' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 11s 259ms/step - loss: 32.6663 - root_mean_squared_error: 5.7154
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.5532 - root_mean_squared_error: 3.2653 - val_loss: 1.7433 - val_root_mean_squared_error: 1.3204

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.1989 - root_mean_squared_error: 1.0950
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5348 - root_mean_squared_error: 1.2379 - val_loss: 1.3203 - val_root_mean_squared_error: 1.1491

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.0970 - root_mean_squared_error: 1.0474
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1628 - root_mean_squared_error: 1.0780 - val_loss: 1.0728 - val_root_mean_squared_error: 1.0357

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.9298 - root_mean_squared_error: 0.9643
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

2025/06/07 11:16:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/06/07 11:17:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 240ms/step - loss: 37.1240 - root_mean_squared_error: 6.0929
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 21.7579 - root_mean_squared_error: 4.5916 - val_loss: 3.0576 - val_root_mean_squared_error: 1.7486

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.5788 - root_mean_squared_error: 1.6059
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5255 - root_mean_squared_error: 1.5878 - val_loss: 2.1513 - val_root_mean_squared_error: 1.4667

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.1479 - root_mean_squared_error: 1.4656
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8522 - root_mean_squared_error: 1.3604 - val_loss: 1.8365 - val_root_mean_squared_error: 1.3552

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 

2025/06/07 11:17:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/06/07 11:17:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



...

2025/06/07 11:17:19 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.

2025/06/07 11:17:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 4/4 [00:26<00:00,  6.72s/trial, best loss: 0.7568786144256592]

2025/06/07 11:17:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


2025/06/07 11:17:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


best parameter: {'lr': np.float64(0.03744173797089241), 'momentum': np.float64(0.2089920840894649)}
best eval rmse : 0.7568786144256592


In [188]:
# Load model as a PyFuncModel.
model_uri = 'runs:/e7ffaa981aea4bfba3a864a90ddbd418/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,0
4656,4.305909
3659,7.492237
907,6.152090
4352,4.245671
3271,5.098396
...,...
2614,6.934031
755,5.981025
518,7.037866
3671,6.543788


In [191]:
import mlflow.pyfunc

model_uri = 'runs:/e7ffaa981aea4bfba3a864a90ddbd418/model'

# Load the model from the run
model = mlflow.pyfunc.load_model(model_uri=model_uri)

# Predict using your input data (e.g., train_x)
predictions = model.predict(train_x)

print(predictions)


92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step
             0
204   4.324694
2413  5.263070
4832  5.384698
623   6.765763
3509  5.451625
...        ...
22    5.134269
509   6.821303
2640  6.481025
1733  5.092724
1810  4.359941

[2938 rows x 1 columns]


In [192]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1749275996156, current_stage='None', description=None, last_updated_timestamp=1749275996156, name='wine-quality', run_id='e7ffaa981aea4bfba3a864a90ddbd418', run_link=None, source='file:///e:/Father%27s_Document/code/MlOps/MlFlow/3_ANN_With_Mlflow/mlruns/843999747342678904/e7ffaa981aea4bfba3a864a90ddbd418/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>